In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-09 00:05:38,814 utils 396 [INFO]    [logger_func] start 


In [225]:
#========================================================================
# CV CHECKER 
# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]].set_index(key)
train = base[~base[target].isnull()]
base_idx = train.reset_index()[[key, target]]

check_model_path = '../stack/0208_002_stack_lgb_lr0.01_235feats_multi157_valterm_5seed_4leaves_iter1031_TERM24_CV3-631440687534257_LB.gz'
check_model = utils.read_pkl_gzip(check_model_path)[[key, 'pred_mean']].set_index(key)

# Indexをそろえる
train = train.join(check_model)

y_train = train[target].values
check_pred = train['pred_mean'].values
check_score = np.sqrt(mean_squared_error(y_train, check_pred))
print(check_score)
#========================================================================


100%|██████████| 3/3 [00:00<00:00, 56.44it/s]

3.7031587732980125


In [12]:
# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, 'no_out_flg']].set_index(key)

# Model Load
best_model_path = '../stack/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz'
best_model = utils.read_pkl_gzip(best_model_path)[[key, 'pred_mean']].set_index(key)

#========================================================================
# Blend
blend1 = utils.read_df_pkl('../stack/0203_102_stack_lgb_lr0.01_156feats_10seed_48leaves_iter1023_OUT29.9500_CV3-629746997354009_LB.gz').set_index(key)['pred_mean']
blend2 = utils.read_df_pkl('../stack/0202_160_stack_lgb_lr0.01_127feats_10seed_48leaves_iter1083_OUT29.9993_CV3-6323025334374535_LB.gz').set_index(key)['pred_mean']
blend3 = utils.read_df_pkl('../stack/dir_01/0130_220_stack_lgb_lr0.01_172feats_10seed_48leaves_iter977_OUT30.0864_CV3-6367844005886467_LB.gz').set_index(key)['pred_mean']
best_model['p1'] = blend1
best_model['p2'] = blend2
best_model['p3'] = blend3
# np.corrcoef(blend1['pred_mean'].values, blend1['p2'].values).min()
# np.corrcoef(blend1['p2'].values, blend1['p3'].values).min()

best_model['pred_mean'] = best_model['pred_mean'].values * 0.4  + best_model['p1'].values * 0.3 + best_model['p2'].values * 0.3
#========================================================================

out_model_path_1 = '../stack/0208_215_stack_lgb_lr0.01_236feats_10seed_70leaves_iter1908_OUT0_CV8-413966707443706_LB.gz'
out_model_path_2 = '../stack/0208_232_stack_lgb_lr0.01_272feats_10seed_70leaves_iter1684_OUT0_CV8-41630196934908_LB.gz'
out_model = utils.read_pkl_gzip(out_model_path_1)[[key, 'pred_mean']].set_index(key)
out_model_2 = utils.read_pkl_gzip(out_model_path_2).set_index(key)['pred_mean']
out_model['p2'] =  out_model_2
out_model['pred_mean'] =  out_model['pred_mean'].values * 0.7 + out_model['p2'].values * 0.3

best_model['no_out_pred'] = out_model['pred_mean']

#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
# clf['clf_pred'] =  clf['pred_mean']
#========================================================================

# Indexをそろえる
base = base.join(best_model)
base['clf_pred'] = clf['clf_pred']
base.head()

100%|██████████| 1/1 [00:00<00:00,  3.31it/s]


,target,no_out_flg,pred_mean,p1,p2,p3,no_out_pred,clf_pred
card_id,,,,,,,,
C_ID_92a2005557,-0.820283,NaN,-0.312293,-0.277248,-0.299641,-0.295437,-0.339001,0.000444
C_ID_3d0044924f,0.392913,NaN,-0.402629,-0.464311,-0.299189,-0.659761,0.276268,0.007831
C_ID_d639edf6cd,0.688056,NaN,0.740270,0.764349,0.759806,0.823495,0.770552,0.004074
C_ID_186d6a6901,0.142495,NaN,0.134626,0.101346,0.134031,0.057771,0.170184,0.000797
C_ID_cdbd2c0db2,-0.159749,1.0,-0.184077,-0.222948,-0.242942,-0.158849,-0.238870,0.000251


In [13]:
score_list = []
ratio_list = []
min_score = 100
pred = 'prediction'
train = base[~base[target].isnull()]
y_train = train[target].values
train[pred] = train['pred_mean']
train[pred] = train['no_out_pred']
for thres in np.arange(0.005, 0.01, 0.0002):
# for thres in np.arange(0.01, 0.7, 0.005):
    
    length = train.loc[train['clf_pred']> thres].shape[0]
    train.loc[train['clf_pred']< thres, pred] = train.loc[train['clf_pred']< thres, 'no_out_pred']
    train.loc[train['clf_pred']>=thres, pred] = train.loc[train['clf_pred']>=thres, 'pred_mean']
#     train.loc[train['clf_pred']> thres, pred] = train.loc[train['clf_pred']> thres, 'no_out_pred']
#     train.loc[train['clf_pred']<=thres, pred] = train.loc[train['clf_pred']<=thres, 'pred_mean']
    train.loc[train['no_out_flg']==1, pred] =  train.loc[train['no_out_flg']==1, 'no_out_pred']
#     train.loc[train[pred].isnull(), pred] = train.loc[train[pred].isnull(), 'pred_mean']
#     train.loc[train[pred].isnull(), pred] = 0
    y_pred = train[pred].values

    score = np.sqrt(mean_squared_error(y_train, y_pred))
    if score<min_score:
        min_score = score
        best_thres = thres
        score_list.append(min_score)
        ratio_list.append(best_thres)
    print(f"Best Score: {min_score} | Best Thres: {best_thres} | LENGTH: {length}")

Best Score: 3.6153113550050735 | Best Thres: 0.005 | LENGTH: 55634
Best Score: 3.615208998628397 | Best Thres: 0.0052 | LENGTH: 54160
Best Score: 3.6151732581371983 | Best Thres: 0.005399999999999999 | LENGTH: 52757
Best Score: 3.6151732581371983 | Best Thres: 0.005399999999999999 | LENGTH: 51425
Best Score: 3.6151732581371983 | Best Thres: 0.005399999999999999 | LENGTH: 50151
Best Score: 3.6151732581371983 | Best Thres: 0.005399999999999999 | LENGTH: 48984
Best Score: 3.615097505525147 | Best Thres: 0.006199999999999998 | LENGTH: 47823
Best Score: 3.614979161373816 | Best Thres: 0.006399999999999998 | LENGTH: 46760
Best Score: 3.6149772123168975 | Best Thres: 0.006599999999999997 | LENGTH: 45788
Best Score: 3.6149772123168975 | Best Thres: 0.006599999999999997 | LENGTH: 44789
Best Score: 3.6149287406085997 | Best Thres: 0.006999999999999997 | LENGTH: 43873
Best Score: 3.614873608780854 | Best Thres: 0.007199999999999996 | LENGTH: 43043
Best Score: 3.614873608780854 | Best Thres: 0.007

In [15]:
blend_num = 3
out_blend_num = 2
thres = 0.0072
base.loc[base['clf_pred']< thres, pred] = base.loc[base['clf_pred']< thres, 'no_out_pred']
base.loc[base['clf_pred']>=thres, pred] = base.loc[base['clf_pred']>=thres, 'pred_mean']
base.loc[base['no_out_flg']==1, pred] =  base.loc[base['no_out_flg']==1, 'no_out_pred']
display(base.head())

submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
submit[target] = base[pred]
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
submit.to_csv(f'../submit/{start_time[4:12]}_submit_outlier_processing_blend{blend_num}_0.4_0.3_0.3_out_blend{out_blend_num}_0.7_0.3_CV{min_score}_LB')
display(submit.head())

,target,no_out_flg,pred_mean,p1,p2,p3,no_out_pred,clf_pred,prediction
card_id,,,,,,,,,
C_ID_92a2005557,-0.820283,NaN,-0.312293,-0.277248,-0.299641,-0.295437,-0.339001,0.000444,-0.339001
C_ID_3d0044924f,0.392913,NaN,-0.402629,-0.464311,-0.299189,-0.659761,0.276268,0.007831,-0.402629
C_ID_d639edf6cd,0.688056,NaN,0.740270,0.764349,0.759806,0.823495,0.770552,0.004074,0.770552
C_ID_186d6a6901,0.142495,NaN,0.134626,0.101346,0.134031,0.057771,0.170184,0.000797,0.170184
C_ID_cdbd2c0db2,-0.159749,1.0,-0.184077,-0.222948,-0.242942,-0.158849,-0.238870,0.000251,-0.238870


,target
card_id,
C_ID_0ab67a22ab,-1.659109
C_ID_130fd0cbdd,-0.203953
C_ID_b709037bc5,-1.298058
C_ID_d27d835a9f,-0.052482
C_ID_2b5e3df5c2,-1.371860


In [14]:
#========================================================================
# Blendのスコアを見る
#========================================================================

score_list = []
ratio_list = []
blend_path_list = glob.glob('../stack/*.gz')

# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]].set_index(key)
train = base[~base[target].isnull()]
base_idx = train.reset_index()[[key, target]]

best_model_path = '../stack/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz'
best_model = utils.read_pkl_gzip(best_model_path)[[key, 'pred_mean']].set_index(key)

# Indexをそろえる
train = train.join(best_model)

y_train = train[target].values
best_pred = train['pred_mean'].values
best_score = np.sqrt(mean_squared_error(y_train, best_pred))
print(f"Best Score: {best_score}")

for i, path in enumerate(blend_path_list):
    tmp = utils.read_pkl_gzip(path)
    tmp = base_idx.merge(tmp, how='inner', on=key)
    
    if 'pred_mean' in tmp.columns:
        tmp_pred = tmp['pred_mean'].values
    else:
        tmp_pred = tmp['prediction'].values
    del tmp
    gc.collect()
    
    tmp_pred = tmp_pred[:len(train)]
    tmp_score_list = []
    min_score = 100
    for ratio in np.arange(0.1, 1.0, 0.1):
        stack_pred = best_pred * ratio + tmp_pred * (1-ratio)
        y_pred = stack_pred[:len(y_train)]
    
        score = np.sqrt(mean_squared_error(y_train, y_pred))
        if score<min_score:
            min_score = score
            min_ratio = ratio
            
    score_list.append(min_score)
    ratio_list.append(min_ratio)
    
    if i%10==0:
#         print(f"BEST: {np.min(score_list)} | BEST RATIO: {ratio_list[np.argmin(score_list)]} | WORST: {np.max(score_list)} | WORST RATIO: {ratio_list[np.argmax(score_list)]}")
        print(f"BEST: {np.min(score_list)} | BEST RATIO: {ratio_list[np.argmin(score_list)]} | PATH: {blend_path_list[np.argmin(score_list)]}")
    
print("Complete!!")

SystemExit: 

In [19]:
#========================================================================
# Stack同士の相関をみる
#========================================================================

corr_list = []
corr_path_list = []
blend_path_list = glob.glob('../stack/*LB.gz')

# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)

best_model_path = '../stack/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz'
best_model = utils.read_pkl_gzip(best_model_path)[[key, 'pred_mean']].set_index(key)

# Indexをそろえる
base = base.join(best_model)

Y = base[target].values
best_pred = base['pred_mean'].values
# base.drop([target, 'pred_mean'], axis=1, inplace=True)
# best_score = np.sqrt(mean_squared_error(y_train, best_pred))
# print(f"Best Score: {best_score}")

for i, path in enumerate(blend_path_list):
    tmp = utils.read_pkl_gzip(path).set_index(key)
    if tmp.shape[0] != best_pred.shape[0]:
        continue
    if path.count('TERM'):
        continue
    corr_path_list.append(path)
    
    if 'pred_mean' in tmp.columns:
        base['p2'] = tmp['pred_mean']
    else:
        base['p2'] = tmp['prediction']
    del tmp
    gc.collect()
    
    corr = np.corrcoef(base['pred_mean'].values, base['p2'].values).min()
    
    corr_list.append(corr)
    print(f"Corr: {corr} | PATH: {path}")
    
print("Complete!!")
df_corr = pd.DataFrame([corr_path_list, corr_list]).T
df_corr.columns = ['path', 'corr']
df_corr.sort_values(by='corr', inplace=True)
df_corr['path'] = df_corr['path'].map(lambda x: re.search(r'CV([^/.]*).gz', x).group(1))
df_corr.head()

100%|██████████| 3/3 [00:00<00:00, 55.31it/s]


Corr: 0.994690355006105 | PATH: ../stack/0205_223_stack_lgb_lr0.01_219feats_10seed_68leaves_iter1237_OUT29.8735_CV3-6239445655263887_LB.gz
Corr: 0.9980713620000645 | PATH: ../stack/0206_164_stack_lgb_lr0.01_243feats_10seed_70leaves_iter1180_OUT29.8251_CV3-620840720122172_LB.gz
Corr: 0.9896210053966104 | PATH: ../stack/0205_114_stack_lgb_lr0.01_204feats_10seed_68leaves_iter1195_OUT29.8844_CV3-624177379034247_LB.gz
Corr: 0.9771769369842552 | PATH: ../stack/0203_130_stack_lgb_lr0.01_191feats_10seed_48leaves_iter1035_OUT29.9618_CV3-6297689094405143_LB.gz
Corr: 0.9893141386217681 | PATH: ../stack/0205_101_stack_lgb_lr0.01_219feats_10seed_68leaves_iter1242_OUT29.8767_CV3-623378898708623_LB.gz
Corr: 0.9775481654815654 | PATH: ../stack/0203_102_stack_lgb_lr0.01_156feats_10seed_48leaves_iter1023_OUT29.9500_CV3-629746997354009_LB.gz
Corr: 0.9899342370483633 | PATH: ../stack/0205_085_stack_lgb_lr0.01_222feats_10seed_68leaves_iter1235_OUT29.8850_CV3-624161605563324_LB.gz
Corr: 0.9970947226422697 |

,path,corr
99,8-413966707443706_LB,0.521283
62,3-6341660938871465_LB,0.959595
18,3-633586049275204_LB,0.960418
37,3-633783290587382_LB,0.962541
73,3-6323025334374535_LB,0.96404


In [33]:
df_corr.sort_values(by='path')

,path,corr
88,3-61968992617503_LB,0.995968
90,3-619940607536087_LB,0.996793
67,3-6200465394024355_LB,0.997118
71,3-6201274184528236_LB,0.99772
81,3-620152638019415_LB,0.997384
44,3-620313558996382_LB,0.997375
20,3-6203797624454666_LB,0.997513
85,3-62047789146233_LB,0.997049
58,3-620579654844301_LB,0.99706
96,3-620630122009932_LB,0.997143


In [ ]:
# Data Load
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]].set_index(key)
train = base[~base[target].isnull()]

# Model Load
best_model_path = '../stack/0206_125_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1164_OUT29.8269_CV3-6215750935280235_LB.gz'
out_model_path = '../stack/0208_210_stack_lgb_lr0.01_236feats_10seed_70leaves_iter1908_OUT0_CV8-413966707443706_LB.gz'
best_model = utils.read_pkl_gzip(best_model_path)[[key, 'pred_mean']].set_index(key)
out_model = utils.read_pkl_gzip(out_model_path)[[key, 'pred_mean']].set_index(key)
best_model['no_out_pred'] = out_model['pred_mean']

#========================================================================
# Classifier
clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
clf['pred_mean_2'] = clf_2['pred_mean']
clf['clf_pred'] =  clf['pred_mean'].values*0.6 + clf['pred_mean_2'].values*0.4
#========================================================================

# Indexをそろえる
train = train.join(best_model)
train['clf_pred'] = clf['clf_pred']
train.head()